# 🚀 KubeSentiment: Getting Started

Welcome to **KubeSentiment** - a production-ready sentiment analysis microservice! This notebook will guide you through the basics of using this MLOps sentiment analysis service.

## 📋 What is KubeSentiment?

KubeSentiment is a comprehensive MLOps solution that provides:
- **Real-time sentiment analysis** using DistilBERT
- **Production-ready API** built with FastAPI
- **Kubernetes deployment** with auto-scaling
- **Comprehensive monitoring** with Prometheus & Grafana
- **Performance benchmarking** tools
- **ONNX optimization** for improved performance

## 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand the KubeSentiment architecture
2. Learn how to interact with the API
3. Perform basic sentiment analysis
4. Understand the response format
5. Explore error handling

## 🏗️ Architecture Overview

KubeSentiment follows a microservice architecture:

```mermaid
graph TB
    A[Client] --> B[API Gateway]
    B --> C[Sentiment Service]
    C --> D[Hugging Face Model]
    C --> E[Prediction Cache]
    
    F[Prometheus] --> G[Metrics]
    H[Grafana] --> G
    I[AlertManager] --> J[Notifications]
```

### Key Components:
- **FastAPI Backend**: High-performance async API
- **DistilBERT Model**: Pre-trained sentiment analysis model
- **LRU Cache**: Performance optimization for repeated predictions
- **Prometheus Metrics**: Real-time monitoring
- **Kubernetes**: Container orchestration and scaling

## 📦 Setup and Dependencies

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install required packages for this notebook
# Note: This cell might take a few minutes to run
!pip install requests httpx pandas matplotlib seaborn plotly

# Import required libraries
import requests
import httpx
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Environment setup complete!")

## 🌐 Connecting to the API

KubeSentiment provides a REST API. Let's first check if the service is running and explore the available endpoints.

In [ ]:
# Configuration
API_BASE_URL = "http://localhost:8000"  # Change this if your service runs on a different port

def check_service_health(base_url: str) -> Dict[str, Any]:
    """Check if the sentiment analysis service is healthy."""
    try:
        response = requests.get(f"{base_url}/health", timeout=10)
        return {
            "status_code": response.status_code,
            "healthy": response.status_code == 200,
            "data": response.json() if response.status_code == 200 else None,
            "error": None
        }
    except Exception as e:
        return {
            "status_code": None,
            "healthy": False,
            "data": None,
            "error": str(e)
        }

# Check service health
health_status = check_service_health(API_BASE_URL)

if health_status["healthy"]:
    print("✅ Service is healthy!")
    print(f"📊 Status: {health_status['data']}")
else:
    print("❌ Service is not available")
    print(f"🔍 Error: {health_status['error']}")
    print("\n💡 Make sure the service is running:")
    print("   docker run -d -p 8000:8000 sentiment-service:latest")
    print("   # or")
    print("   python -m uvicorn app.main:app --host 0.0.0.0 --port 8000 --reload")

## 📊 Model Information

Let's explore what model is being used and its capabilities.

In [ ]:
def get_model_info(base_url: str) -> Dict[str, Any]:
    """Get information about the loaded model."""
    try:
        response = requests.get(f"{base_url}/model-info", timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"HTTP {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Get model information
if health_status["healthy"]:
    model_info = get_model_info(API_BASE_URL)
    
    if "error" not in model_info:
        print("🤖 Model Information:")
        print(f"📝 Model Name: {model_info.get('model_name', 'N/A')}")
        print(f"✅ Loaded: {model_info.get('is_loaded', False)}")
        print(f"🚀 Ready: {model_info.get('is_ready', False)}")
        print(f"💾 Cache Size: {model_info.get('cache_stats', {}).get('cache_size', 0)}")
        print(f"🔧 PyTorch Version: {model_info.get('torch_version', 'N/A')}")
        print(f"🖥️ CUDA Available: {model_info.get('cuda_available', False)}")
    else:
        print(f"❌ Error getting model info: {model_info['error']}")
else:
    print("⏭️ Skipping model info check - service not available")

## 🎯 Sentiment Analysis

Now let's perform some sentiment analysis! The API accepts text and returns sentiment predictions.

In [ ]:
def analyze_sentiment(base_url: str, text: str) -> Dict[str, Any]:
    """Analyze sentiment of the given text."""
    try:
        payload = {"text": text}
        response = requests.post(
            f"{base_url}/predict", 
            json=payload, 
            timeout=30
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            return {
                "error": f"HTTP {response.status_code}", 
                "details": response.text,
                "text": text
            }
    except Exception as e:
        return {"error": str(e), "text": text}

# Test cases with different sentiments
test_texts = [
    "I absolutely love this product! It's amazing and works perfectly.",
    "This is terrible. I'm very disappointed with the quality.",
    "The service was okay, nothing special but it worked.",
    "Outstanding customer support and excellent features!",
    "Worst experience ever. Complete waste of money.",
    "It's decent, does what it needs to do without issues."
]

print("🎯 Sentiment Analysis Results:")
print("=" * 60)

if health_status["healthy"]:
    results = []
    for text in test_texts:
        result = analyze_sentiment(API_BASE_URL, text)
        
        if "error" not in result:
            print(f"📝 Text: {text[:50]}...")
            print(f"😊 Sentiment: {result['label']}")
            print(f"📊 Confidence: {result['score']:.3f}")
            print(f"⚡ Inference Time: {result['inference_time_ms']:.2f}ms")
            print(f"📏 Text Length: {result['text_length']} chars")
            print("-" * 40)
            
            results.append({
                "text": text,
                "label": result["label"],
                "score": result["score"],
                "inference_time_ms": result["inference_time_ms"],
                "text_length": result["text_length"]
            })
        else:
            print(f"❌ Error analyzing: {text[:30]}...")
            print(f"   Error: {result['error']}")
            print("-" * 40)
else:
    print("⏭️ Skipping sentiment analysis - service not available")

## 📈 Analyzing Results

Let's create some visualizations to understand our sentiment analysis results better.

In [ ]:
# Create DataFrame from results
if 'results' in locals() and results:
    df = pd.DataFrame(results)
    
    # Display results table
    print("📊 Sentiment Analysis Summary:")
    display(df)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Sentiment Analysis Results', fontsize=16)
    
    # 1. Sentiment distribution
    sentiment_counts = df['label'].value_counts()
    axes[0, 0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Sentiment Distribution')
    
    # 2. Confidence scores
    colors = ['green' if label == 'POSITIVE' else 'red' if label == 'NEGATIVE' else 'blue' 
              for label in df['label']]
    axes[0, 1].bar(range(len(df)), df['score'], color=colors)
    axes[0, 1].set_title('Confidence Scores')
    axes[0, 1].set_xlabel('Sample')
    axes[0, 1].set_ylabel('Confidence')
    axes[0, 1].set_ylim(0, 1)
    
    # 3. Inference time distribution
    axes[1, 0].hist(df['inference_time_ms'], bins=10, alpha=0.7, color='skyblue')
    axes[1, 0].set_title('Inference Time Distribution')
    axes[1, 0].set_xlabel('Time (ms)')
    axes[1, 0].set_ylabel('Frequency')
    
    # 4. Text length vs confidence
    scatter_colors = ['green' if label == 'POSITIVE' else 'red' if label == 'NEGATIVE' else 'blue' 
                      for label in df['label']]
    axes[1, 1].scatter(df['text_length'], df['score'], c=scatter_colors, alpha=0.6)
    axes[1, 1].set_title('Text Length vs Confidence')
    axes[1, 1].set_xlabel('Text Length')
    axes[1, 1].set_ylabel('Confidence Score')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n📈 Summary Statistics:")
    print(f"📊 Total predictions: {len(df)}")
    print(f"😊 Positive sentiments: {len(df[df['label'] == 'POSITIVE'])}")
    print(f"😢 Negative sentiments: {len(df[df['label'] == 'NEGATIVE'])}")
    print(f"😐 Neutral sentiments: {len(df[df['label'] == 'NEUTRAL'])}")
    print(f"⚡ Average inference time: {df['inference_time_ms'].mean():.2f}ms")
    print(f"📏 Average text length: {df['text_length'].mean():.0f} characters")
    print(f"🎯 Average confidence: {df['score'].mean():.3f}")
    
else:
    print("⏭️ No results to analyze - service not available or no predictions made")

## 🚨 Error Handling

Let's explore how the API handles various error conditions.

In [ ]:
# Test error scenarios
error_test_cases = [
    {"text": "", "description": "Empty text"},
    {"text": "   ", "description": "Whitespace only"},
    {"text": "a" * 10000, "description": "Text too long"},
    {"text": "This is normal text.", "description": "Valid text (should work)"}
]

print("🚨 Error Handling Test Results:")
print("=" * 50)

if health_status["healthy"]:
    for test_case in error_test_cases:
        result = analyze_sentiment(API_BASE_URL, test_case["text"])
        
        print(f"🧪 Test: {test_case['description']}")
        
        if "error" in result:
            print(f"   ❌ Expected error: {result['error']}")
        else:
            print(f"   ✅ Success: {result['label']} (confidence: {result['score']:.3f})")
        print("-" * 30)
else:
    print("⏭️ Skipping error handling tests - service not available")

## 📊 Performance Metrics

Let's check the service's performance metrics.

In [ ]:
def get_metrics(base_url: str) -> Dict[str, Any]:
    """Get service performance metrics."""
    try:
        response = requests.get(f"{base_url}/metrics-json", timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"HTTP {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Get performance metrics
if health_status["healthy"]:
    metrics = get_metrics(API_BASE_URL)
    
    if "error" not in metrics:
        print("📊 Performance Metrics:")
        print(f"🔧 PyTorch Version: {metrics.get('torch_version', 'N/A')}")
        print(f"🖥️ CUDA Available: {metrics.get('cuda_available', 'N/A')}")
        print(f"💾 Memory Allocated: {metrics.get('cuda_memory_allocated_mb', 0)} MB")
        print(f"📈 Memory Reserved: {metrics.get('cuda_memory_reserved_mb', 0)} MB")
        print(f"🎮 CUDA Devices: {metrics.get('cuda_device_count', 0)}")
    else:
        print(f"❌ Error getting metrics: {metrics['error']}")
else:
    print("⏭️ Skipping metrics check - service not available")

## 🎉 Conclusion

Congratulations! You've successfully:

1. ✅ **Connected to the KubeSentiment API**
2. ✅ **Explored the model capabilities**
3. ✅ **Performed sentiment analysis**
4. ✅ **Analyzed results with visualizations**
5. ✅ **Tested error handling**
6. ✅ **Checked performance metrics**

## 🚀 Next Steps

Now that you understand the basics, explore these advanced topics:

- **[02_model_exploration.ipynb](02_model_exploration.ipynb)**: Deep dive into the sentiment model
- **[03_api_testing.ipynb](03_api_testing.ipynb)**: Comprehensive API testing
- **[04_benchmarking_analysis.ipynb](04_benchmarking_analysis.ipynb)**: Performance benchmarking
- **[05_monitoring_metrics.ipynb](05_monitoring_metrics.ipynb)**: Advanced monitoring
- **[06_development_workflow.ipynb](06_development_workflow.ipynb)**: Development workflows
- **[07_deployment_guide.ipynb](07_deployment_guide.ipynb)**: Production deployment

## 📚 Additional Resources

- **[API Documentation](http://localhost:8000/docs)**: Interactive Swagger UI
- **[Main README](../README.md)**: Complete project documentation
- **[Architecture Guide](../docs/architecture.md)**: System design overview
- **[Benchmarking Guide](../docs/BENCHMARKING.md)**: Performance testing

---

**Happy analyzing! 🎯**